In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Create a tensor with variable length sequences and pads (25)
seqs = torch.LongTensor([[0, 1, 2, 3, 25, 25, 25],
                         [4, 5, 25, 25, 25, 25, 25],
                         [6, 7, 8, 9, 10, 11, 25]])

# Store lengths of the actual sequences, ignoring padding
# These are the points up to which we want the RNN to process the sequence
seq_lens = torch.LongTensor([4, 2, 6])

# Sort by decreasing lengths
seq_lens, sort_ind = seq_lens.sort(dim=0, descending=True)
seqs = seqs[sort_ind]

# Create an embedding layer, with 0 vectors for the pads
embeds = nn.Embedding(26, 10, padding_idx=25)

# Create an LSTM layer
lstm = nn.LSTM(10, 50, bidirectional=False, batch_first=True)

# WITHOUT DYNAMIC BATCHING

embeddings = embeds(seqs)
out_static, _ = lstm(embeddings)

# The number of timesteps in the output will be the same as the total padded timesteps in the input,
# since the LSTM computed over the pads
assert out_static.size(1) == embeddings.size(1)

# Look at the output at a timestep that we know is a pad
print(out_static[1, -1])

# WITH DYNAMIC BATCHING

# Pack the sequence
print(embeddings)
packed_seqs = pack_padded_sequence(embeddings, seq_lens.tolist(), batch_first=True)
print(packed_seqs.data)

# To execute the LSTM over only the valid timesteps
out_dynamic, _ = lstm(packed_seqs)

# Use the inverse function to re-pad it
out_dynamic, lens = pad_packed_sequence(out_dynamic, batch_first=True)

# Note that since we re-padded it, the total padded timesteps will be the length of the longest sequence (6)
assert out_dynamic.size(1) != embeddings.size(1)
print(out_dynamic.shape)

# Look at the output at a timestep that we know is a pad
print(out_dynamic[1, -1])

# It's all zeros!

#########################################################

# So, what does pack_padded_sequence do?
# It removes pads, flattens by timestep, and keeps track of effective batch_size at each timestep

# The RNN computes only on the effective batch size "b_t" at each timestep
# This is why we sort - so the top "b_t" rows at timestep "t" are aligned with the top "b_t" outputs from timestep "t-1"

# Consider the original encoded sequences (sorted)
print(seqs)

# Let's pack it
packed_seqs = pack_padded_sequence(seqs, seq_lens, batch_first=True)

# The result of pack_padded_sequence() is a tuple containing the flattened tensor and the effective batch size at each timestep
# Here's the flattened tensor with pads removed
print(packed_seqs[0])
# You can see it's flattened timestep-wise
# Since pads are removed, the total datapoints are equal to the number of valid timsteps
assert packed_seqs[0].size(0) == sum(seq_lens.tolist())

# Here's the effective batch size at each timestep
print(packed_seqs[1])
# If you look at the original encoded sequences, you can see this is true
print(seqs)


tensor([-0.0609, -0.0287,  0.0017, -0.1018,  0.0162, -0.0309,  0.0448,  0.0186,
        -0.0874,  0.0060, -0.0332, -0.0179,  0.0703,  0.0343, -0.0756,  0.0776,
        -0.0547, -0.0124, -0.0628,  0.0992, -0.0947, -0.0011, -0.0209, -0.1573,
         0.0162,  0.0056,  0.0363, -0.0659, -0.0826, -0.0492,  0.0555, -0.0085,
        -0.0052,  0.0211,  0.0050, -0.0451,  0.0675, -0.0343,  0.0707,  0.1128,
         0.1015, -0.0150, -0.0642, -0.0365,  0.0472,  0.0280,  0.0412, -0.0561,
        -0.0222, -0.0039], grad_fn=<SelectBackward0>)
tensor([[ 0.1021,  0.2622,  0.8161, -0.4219,  0.8288,  1.3201, -0.3965,  1.2111,
         -2.2316,  0.2475],
        [ 0.6104,  0.0099, -0.7641, -0.5004, -1.2632,  2.4780, -0.7719,  0.2716,
          0.3371,  1.6649],
        [-0.3771,  0.6596,  0.7194,  0.2166,  0.3540, -0.8103, -0.2909,  1.6902,
          1.4472,  0.0470],
        [ 0.0344, -0.3339, -0.2957, -1.2068, -0.1760, -0.2996, -0.2708, -1.9583,
         -0.4160,  0.6845],
        [-0.3454,  0.5702, -1.